# Machine Translation datasets


### Datasets listos para su uso

En [Hugging Face](https://huggingface.co/datasets) hay multiples datasets listos para su uso. Para el caso particular de la traducción de idiomas (Machine Translation) podemos filtrar los datasets que se ajusten a nuestra necesidad. Por ejemplo, vamos [a utilizar un dataset](https://huggingface.co/datasets/Iker/Document-Translation-en-es) que se define como:
*"Este conjunto de datos contiene 10533 artículos de noticias de [ELiRF/dacsa](https://huggingface.co/datasets/ELiRF/dacsa) traducidos del español al inglés utilizando GPT-3.5-turbo. El conjunto de datos está pensado para entrenar un modelo de traducción de textos del inglés al español y viceversa. El conjunto de datos también es útil para evaluar modelos de traducción automática a nivel de documento."*
A continuación vemos como se puede descargar dicho dataset desde hugging face y dividirlo en dos conjuntos, de entrenamiento y de pruebas/validación.

#### (Des)Cargar un dataset
El método `load_dataset` se utiliza para cargar datasets del catálogo de Hugging Face o desde fuentes locales. Este método soporta varios parámetros que permiten personalizar cómo se carga el dataset.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("Iker/Document-Translation-en-es", split="train")

print("Tipo de dato: ", type(dataset))
print("Tamaño del dataset: ", len(dataset))
print("--- Conjunto de pruebas ---")
print(dataset)

In [ ]:
print("--- Primera fila es ---")
print(dataset["es"][0])

In [ ]:
print("--- Primera fila en ---")
print(dataset["en"][0])

Existen diferentes parámetros que se pueden especificar que es importante conocer:

##### Parámetro split

El parámetro `split` especifica qué parte del dataset se quiere cargar. Normalmente, toma como valor `train` para indicar datos de entrenamiento, `test` para indicar datos de prueba y `validation` para indicar datos de validación (si el dataset tiene esta partición); además de otros valores menos frecuentes.

In [ ]:
def load_ds(name, split_type):
    try:
        ds = load_dataset(name, split=split_type)
        print(name, " tamaño del dataset de ",split_type," ", len(ds))
    except Exception as e:
        print(e)

# Prueba con dataset Iker/Document-Translation-en-es
load_ds("Iker/Document-Translation-en-es", split_type="train")
load_ds("Iker/Document-Translation-en-es", split_type="test")
load_ds("Iker/Document-Translation-en-es", split_type="validation")
# Prueba con dataset de imdb
load_ds("imdb", split_type="train")
load_ds("imdb", split_type="test")
load_ds("imdb", split_type="validation")

Como se puede observar, el valor de `split` depende de como se ha preparado el dataset. En el caso de `Iker/Document-Translation-en-es` solo hay un split de training mientras que en `imdb` existe de training, test, pero no de validación y en su lugar existe uno llamado `unsupervised`.

Además, se pueden crear subconjuntos personalizados añadiendo los carácteres `[]` al conjunto de `split`

In [ ]:
# Cargar solo el 10% de los datos de entrenamiento
subset_dataset = load_dataset("Iker/Document-Translation-en-es", split="train[:10%]")
print("Tamaño del subset (10% de entrenamiento): ", len(subset_dataset))

In [ ]:
# Cargar ejemplos desde el índice 100 hasta el 200
range_dataset = load_dataset("Iker/Document-Translation-en-es", split="train[100:200]")
print("Tamaño del subset (índices 100-200): ", len(range_dataset))

Además se pueden combinar distintas particiones:

In [ ]:
# Combinar datos de entrenamiento y prueba
combined_dataset = load_dataset("imdb", split="train+test")
print("Tamaño del dataset combinado: ",len(combined_dataset))

##### Parámetro `cache_dir`

Este parámetro es *especialmente importante para datasets grandes* ya que permite guardar una copia del dataset en local para poder utilizarlo a posteriori sin tener que volver a descargarlo.

In [ ]:
import os

print("Existe dataset: ", os.path.exists("./data/iker"))
# Almacenar el dataset en un directorio específico
cached_dataset = load_dataset("Iker/Document-Translation-en-es", split="train", cache_dir="./data/iker")
print("Existe dataset: ", os.path.exists("./data/iker"))

Si el fragmento anterior se ejecuta dos veces, la segunda se ve como el dataset ya no es descargado.

##### Parámetro `data_files`

Este parámetro se utiliza para indicar como dataset un fichero local el cuál será cargado. Los datasets locales  puede estar escritos en diversos formatos: `csv`, `json`, o `parquet`.

In [ ]:
# Cargar un dataset desde un archivo CSV local
local_dataset = load_dataset("csv", data_files={"train": "./provided/sample-ds.csv", "test": "./provided/sample-ds.csv"}, split="train")
print(f"Tamaño del dataset local: {len(local_dataset)}")

#### Cargar un dataset desde memoria

Por último, es posible cargar un dataset desde una variable que tengamos en el código. Para ello se usa el método estático `Dataset.from_dict()` que carga un diccionario de python.

In [ ]:
from datasets import Dataset
import io

# Importante el formato del dic !
raw_dataset_json = { "translations": [
    {"español": "Hola, ¿cómo estás?", "latin": "Salve, quid agis?"},
    {"español": "Buenos días", "latin": "Bonum mane"},
    {"español": "Buenas noches", "latin": "Bonum nocte"},
    {"español": "Gracias por tu ayuda", "latin": "Gratias tibi ago pro auxilio tuo"},
    {"español": "¿Dónde está la biblioteca?", "latin": "Ubi est bibliotheca?"},
    {"español": "Tengo hambre", "latin": "Esurio"},
    {"español": "Tengo sed", "latin": "Sitiens sum"},
    {"español": "Me gusta leer libros", "latin": "Libros legere mihi placet"},
    {"español": "¿Cuánto cuesta esto?", "latin": "Quanti constat hoc?"},
    {"español": "Por favor, repítelo", "latin": "Quaeso, iterum dic"},
    {"español": "Disculpa, no entiendo", "latin": "Ignosce, non intellego"},
    {"español": "Mi nombre es Juan", "latin": "Nomen meum est Ioannes"},
    {"español": "Feliz cumpleaños", "latin": "Felix natalis"},
    {"español": "Hasta luego", "latin": "Vale"},
    {"español": "Estoy perdido", "latin": "Erravi"},
    {"español": "¿Puedes ayudarme?", "latin": "Potesne me adiuvare?"},
    {"español": "¿Cuál es tu nombre?", "latin": "Quod est nomen tuum?"},
    {"español": "Estoy aprendiendo latín", "latin": "Disco latinam linguam"},
    {"español": "Es un placer conocerte", "latin": "Placuit mihi te convenire"},
    {"español": "Lo siento mucho", "latin": "Multum doleo"},
    {"español": "Estoy cansado", "latin": "Fessus sum"},
    {"español": "Vengo de España", "latin": "Ex Hispania venio"},
    {"español": "Me gusta mucho la música", "latin": "Musica mihi multum placet"},
    {"español": "Quiero ir al mercado", "latin": "Volo ire ad forum"},
    {"español": "Estoy buscando un restaurante", "latin": "Quaero tabernam cauponiam"},
    {"español": "¿Hablas español?", "latin": "Loquerisne Hispanice?"},
    {"español": "Hoy es un buen día", "latin": "Hodie est dies bonus"},
    {"español": "Estoy de acuerdo contigo", "latin": "Assentior tibi"},
    {"español": "La comida está deliciosa", "latin": "Cibus est deliciosus"},
    {"español": "Quiero aprender más", "latin": "Volo plus discere"}
    ]
}

# Load the dataset from the JSON buffer
dataset = Dataset.from_dict(raw_dataset_json)

# Print the dataset
print(dataset)
split_dataset = dataset.train_test_split(train_size=0.5, shuffle=True, seed=42)

train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

print("Tamaño del conjunto de entrenamiento: ", len(train_dataset))
print("Tamaño del conjunto de prueba: ", len(eval_dataset))
print("Tamaño total: ", len(eval_dataset) + len(train_dataset))

#### Preparar un dataset para entrenar y evaluar/validar

Haciendo uso del método `train_test_split` que possen los objetos de tipo `dataset` se pueden generar los datasets para el entrenamiento y la validación. Es importante conocer algunos parámetros:
* `test_size` y `train_size` sirven para indicar el tamaño del conjunto de evaluación/validación y de entrenamiento respectivamente. Aceptan un valor `n` que indica el procentaje de ejemplos (`n<1.0`) o el número exacto a usar (`n>1`)
* shuffle: Determina si los datos deben barajarse antes de dividirlos. Si shuffle=True (por defecto), los datos se distribuyen aleatoriamente entre los subconjuntos. Si es False, los datos se dividen en su orden original.
* seed: Establece la semilla para la aleatoriedad del barajado. Esto permite reproducir divisiones consistentes en diferentes ejecuciones al usar el mismo valor de semilla.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Iker/Document-Translation-en-es", split="train", cache_dir="./data/iker")
split_dataset = dataset.train_test_split(train_size=0.5, shuffle=True, seed=42)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

print("Tamaño del conjunto de entrenamiento: ", len(train_dataset))
print("Tamaño del conjunto de prueba: ", len(eval_dataset))
print("Tamaño total: ", len(eval_dataset) + len(train_dataset))


#### Tarea DS1

Buscar el dataset `okezieowen/english_to_spanish` en hugging face para poder ser usado para traducción (machine translation) entre inglés y español. Escribir el código python necesario para que usando dicho dataset se genere un conjunto de entrenamiento y otro de evaluación/validación. Además, hacer que dicho dataset se guarde en cache local bajo la carpeta `./data/`



### Construir un dataset



Para construir un dataset orientado a la traducción, sabemos que necesitamos construir un `dict` de python donde tengamos una estructura:
````
{ "translations" : [
  { "l1" : "...", "l2" : "....", ... }
 ] }
````
Donde `li` es el idioma en el que está codificada la frase

#### Tarea DS2

Construir un dataset para los idiomas del señor de los anillos utilizando [los subtitulos de las películas del señor de los anillos](https://github.com/howa003/complete-elvish-lotr-subtitles). Preferiblemente, el dataset tiene que por lo menos contener el idioma [Sindarin](https://es.wikipedia.org/wiki/Sindarin#:~:text=El%20sindarin%20(llamado%20tambi%C3%A9n%20%C3%A9lfico,hablada%20en%20la%20Tierra%20Media) (Élfico Gris).



Una posible solución *out of the box* se encuentra implementada en el fichero `./libs/sindarin_builder.py`

In [ ]:
from libs.lotr_ds_builder import build_datasets

train, test = build_datasets()
print(train.keys())
print(train['Sindarin'])